In [2]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/c-sharp.jsonl

In [3]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
ls = []

with open('c-sharp.jsonl') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
            if len(ls) > 200000:
                break
        except:
            pass
len(ls)

200000it [00:12, 15461.84it/s]


198043

In [5]:
client = InferenceClient(
    '', timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [6]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [7]:
!mkdir mixtral-c-sharp
# !rm -rf mixtral-c-sharp/*.json

In [8]:
def answer(q, i):
    filename = f'mixtral-c-sharp/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality C# problem. Present your output in two distinct sections: [Problem Description] and [Solution].

C# snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [9]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [10]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [11]:
answer(*urls[0])

In [12]:
with open('mixtral-c-sharp/0.json') as fopen:
    print(json.loads(fopen.read()))

 **[Problem Description]**

Create a class named `Banner` in C# that accepts a message as input during instantiation. The class should have a method named `display` that prints the message to the console in the style of a banner, using asterisks (`*`) to surround the message. The banner should have a top and bottom border that is 5 asterisks thick, while the left and right borders should be only 3 asterisks thick. The message itself should be centered within the banner.

The `display` method should follow these formatting rules:
1. The banner's borders should always be 5 asterisks thick at the top and bottom.
2. The message should be centered in the banner, with an equal number of asterisks on both sides.
3. If the message is too long to fit within the banner's width, it should be truncated with an ellipsis (`...`).
4. The number of asterisks on the left and right sides of the banner should be 3, regardless of the message length.

**[Solution]**

```csharp
public class Banner
{
    pri

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  2%|█▉                                                                                                | 3942/198043 [4:01:22<530:27:30,  9.84s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  7%|██████▊                                                                                         | 13985/198043 [13:20:32<112:31:42,  2.20s/it]